In [2]:
import psycopg2
import csv
conn = psycopg2.connect(dbname="postgres", user="postgres", password = "abc123")
cur = conn.cursor()
conn.autocommit = True

In [3]:
cur.execute("CREATE DATABASE crime_db;")
conn.close()

DuplicateDatabase: database "crime_db" already exists


In [4]:
conn = psycopg2.connect(dbname="crime_db", user="postgres", password = "abc123")
conn.autocommit = True
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

DuplicateSchema: schema "crimes" already exists


In [5]:
with open('boston.csv', 'r') as f:
    reader = csv.reader(f)
    col_headers = next(reader)

In [6]:
def get_col_set(csv_file, col_index):
    import csv
    values = set()
    with open(csv_file, 'r') as f:
        next(f)
        reader = csv.reader(f)
        for row in reader:
            values.add(row[col_index])
    return values


In [7]:
for i in range(len(col_headers)):
    values = get_col_set("boston.csv", i)
    print(col_headers[i], len(values))

incident_number 298329
offense_code 219
description 239
date 1177
day_of_the_week 7
lat 18177
long 18177


In [9]:
descriptions = get_col_set('boston.csv',2)

In [10]:
max_len = 0 
for description in descriptions:
    max_len = max(max_len,len(description))
print(max_len)

58


In [11]:
# create the enumerated datatype for representing the weekday
cur.execute("""
    CREATE TYPE weekday AS ENUM ('Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday');
""")
# create the table
cur.execute("""
    CREATE TABLE crimes.boston_crimes (
        incident_number INTEGER PRIMARY KEY,
        offense_code INTEGER,
        description VARCHAR(100),
        date DATE,
        day_of_the_week weekday,
        lat decimal,
        long decimal
    );
""")

In [12]:
with open("boston.csv") as f:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", f)

In [14]:
cur.execute("SELECT * FROM crimes.boston_crimes")

In [ ]:
print(cur.fetchall())